In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import os
from datetime import datetime

# Initialize MediaPipe solutions
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

def get_avg_color(region):
    avg_color_per_row = np.average(region, axis=0)
    avg_color = np.average(avg_color_per_row, axis=0)
    return avg_color

def bgr_to_hex(bgr):
    r, g, b = int(bgr[2]), int(bgr[1]), int(bgr[0])
    return "#{:02x}{:02x}{:02x}".format(r, g, b).upper()

def sample_feature_color(frame, landmarks, idx, region_size=6):
    h, w, _ = frame.shape
    x = int(landmarks.landmark[idx].x * w)
    y = int(landmarks.landmark[idx].y * h)
    patch = frame[y-region_size:y+region_size, x-region_size:x+region_size]
    if patch.size == 0:
        return None, (x, y)
    avg_bgr = get_avg_color(patch)
    return avg_bgr, (x, y)

def initialize_csv(filename):
    if not os.path.exists(filename):
        with open(filename, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(['Timestamp', 'Eye_R', 'Eye_G', 'Eye_B', 'Eye_Hex', 
                               'Lips_R', 'Lips_G', 'Lips_B', 'Lips_Hex',
                               'Cheek_R', 'Cheek_G', 'Cheek_B', 'Cheek_Hex',
                               'Hair_R', 'Hair_G', 'Hair_B', 'Hair_Hex'])
    return filename

def append_to_csv(filename, feature_colors):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    row_data = [timestamp]
    
    for feature in ['Eye', 'Lips', 'Cheek', 'Hair']:
        if feature in feature_colors and feature_colors[feature] is not None:
            color_bgr = feature_colors[feature]
            r, g, b = int(color_bgr[2]), int(color_bgr[1]), int(color_bgr[0])
            hex_color = bgr_to_hex(color_bgr)
            row_data.extend([r, g, b, hex_color])
        else:
            row_data.extend([None, None, None, None])
    
    with open(filename, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(row_data)

def start_color_analysis():
    csv_filename = "beyonce-features.csv"
    initialize_csv(csv_filename)
    
    cap = cv2.VideoCapture(0)
    
    # Custom drawing specifications with blue color for mesh
    blue_mesh_spec = mp_drawing.DrawingSpec(color=(255, 100, 200), thickness=1, circle_radius=1)
    
    # MediaPipe FaceMesh configuration
    with mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as face_mesh:
        
        recording = False
        show_mesh = True  # Toggle for showing/hiding the facial mesh
        feature_colors = {}
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
                
            # Convert to RGB for MediaPipe
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Process the image
            results = face_mesh.process(image_rgb)
            
            # Make a copy of the frame for visualization
            display_frame = frame.copy()
            
            if results.multi_face_landmarks:
                face_landmarks = results.multi_face_landmarks[0]
                
                # Draw the facial mesh if enabled
                if show_mesh:
                    # Draw tessellation with custom blue color
                    mp_drawing.draw_landmarks(
                        image=display_frame,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=blue_mesh_spec
                    )
                
                features = {
                    'Eye': 468,   # Left iris center
                    'Lips': 13,   # Lower lip center
                    'Cheek': 205, # Left cheek
                    'Hair': 295   # Eyebrow, in place for hair color
                }
                
                # Reset feature_colors for this frame
                feature_colors = {}
                
                y_offset = 20
                for i, (feature, idx) in enumerate(features.items()):
                    color_bgr, (x, y) = sample_feature_color(frame, face_landmarks, idx)
                    if color_bgr is not None:
                        feature_colors[feature] = color_bgr
                        r, g, b = int(color_bgr[2]), int(color_bgr[1]), int(color_bgr[0])
                        hex_color = bgr_to_hex(color_bgr)
                        
                        # Text display
                        text = f"{feature}: RGB({r},{g},{b}) | {hex_color}"
                        cv2.putText(display_frame, text, (10, 30 + y_offset * i), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                        
                        # Color swatch next to face
                        swatch_x = x + 10
                        swatch_y = y - 10
                        cv2.rectangle(display_frame, (swatch_x, swatch_y), (swatch_x + 40, swatch_y + 20), (b, g, r), -1)
                        cv2.rectangle(display_frame, (swatch_x, swatch_y), (swatch_x + 40, swatch_y + 20), (0, 0, 0), 1)
                        
                        # Highlight feature point with bright red (to stand out against blue mesh)
                        cv2.circle(display_frame, (x, y), 3, (255, 255, 255), -1)
            
            # Display recording status
            if recording:
                cv2.putText(display_frame, "Recording: ON (Press 'r' to stop)", (10, display_frame.shape[0] - 20), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            else:
                cv2.putText(display_frame, "Recording: OFF (Press 'r' to start)", (10, display_frame.shape[0] - 20), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                
            # Display CSV status and controls
            cv2.putText(display_frame, f"CSV: {csv_filename}", (10, display_frame.shape[0] - 50), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            cv2.putText(display_frame, "Press 'm' to toggle mesh", (10, display_frame.shape[0] - 80), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
                       
            # Show current frame    
            cv2.imshow('KoreAI Analyzer – Feature Colors', display_frame)
            
            # Handle keyboard input
            key = cv2.waitKey(5) & 0xFF
            if key == 27:  # ESC key
                break
            elif key == ord('r'):  # Toggle recording
                recording = not recording
                if recording:
                    print(f"Recording started. Data will be saved to {csv_filename}")
                else:
                    print(f"Recording stopped. Data saved to {csv_filename}")
            elif key == ord('s') and len(feature_colors) > 0:  # Manual save current frame
                append_to_csv(csv_filename, feature_colors)
                print(f"Snapshot saved to {csv_filename}")
            elif key == ord('m'):  # Toggle mesh visualization
                show_mesh = not show_mesh
                print(f"Face mesh visualization: {'ON' if show_mesh else 'OFF'}")
            
            # If recording and we have detected features, append to CSV
            if recording and results.multi_face_landmarks and len(feature_colors) > 0:
                append_to_csv(csv_filename, feature_colors)
                
    cap.release()
    cv2.destroyAllWindows()
    print(f"Analysis complete. Data saved to {csv_filename}")

if __name__ == "__main__":
    start_color_analysis()

2025-04-05 15:19:34.059 python[1962:18691] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
I0000 00:00:1743891575.427373   18691 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1743891575.437252   49122 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743891575.444915   49122 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743891575.447846   49118 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Face mesh visualization: OFF
Face mesh visualization: ON
Face mesh visualization: OFF
Face mesh visualization: ON
Face mesh visualization: OFF
Face mesh visualization: ON
Face mesh visualization: OFF
Face mesh visualization: ON
Face mesh visualization: OFF
Face mesh visualization: ON
Face mesh visualization: OFF
Face mesh visualization: ON
Face mesh visualization: OFF
Face mesh visualization: ON
Face mesh visualization: OFF
Face mesh visualization: ON
Face mesh visualization: OFF


In [16]:
import pandas as pd

sp_df = pd.read_csv('sp-features.csv')[['Eye_R', 'Eye_G', 'Eye_B', 
                               'Lips_R', 'Lips_G', 'Lips_B',
                               'Cheek_R', 'Cheek_G', 'Cheek_B',
                               'Hair_R', 'Hair_G', 'Hair_B']]

sp_avg = sp_df.mean(axis=0)

sp_avg

Eye_R       34.209302
Eye_G       32.511628
Eye_B       36.825581
Lips_R     110.872093
Lips_G      57.104651
Lips_B      58.406977
Cheek_R    207.418605
Cheek_G    143.069767
Cheek_B    118.081395
Hair_R      59.767442
Hair_G      52.406977
Hair_B      49.406977
dtype: float64